In [1]:
import os, joblib, pandas as pd, matplotlib.pyplot as plt

# Define the input path
input_dir = "../data/final/"

# Load the datasets
X_train = pd.read_csv(os.path.join(input_dir, "training_set.csv"))
X_test = pd.read_csv(os.path.join(input_dir, "testing_set.csv"))

# Prepare features and target
X = X_train.drop(columns=['YIELD'])
y = X_train['YIELD']
X_test_features = X_test.drop(columns=['YIELD'])
y_test = X_test['YIELD']

In [2]:
# Mapping Dictionary
indicator_labels = {
    "WB_CCKP_CDD": "Consecutive Dry Days",
    "WB_CCKP_CDD65": "Cooling Degree Days 65°F",
    "WB_CCKP_CSDI": "Cold Spell Duration Index",
    "WB_CCKP_CWD": "Consecutive Wet Days",
    "WB_CCKP_FD": "Frost Days",
    "WB_CCKP_HD30": "Hot Days >30°C",
    "WB_CCKP_HD35": "Hot Days >35°C",
    "WB_CCKP_HD40": "Hot Days >40°C",
    "WB_CCKP_HD42": "Hot Days >42°C",
    "WB_CCKP_HD45": "Hot Days >45°C",
    "WB_CCKP_HD50": "Hot Days >50°C",
    "WB_CCKP_HDD65": "Heating Degree Days 65°F",
    "WB_CCKP_HI35": "Heat Index >35°C",
    "WB_CCKP_HI37": "Heat Index >37°C",
    "WB_CCKP_HI39": "Heat Index >39°C",
    "WB_CCKP_HI41": "Heat Index >41°C",
    "WB_CCKP_HURS": "Relative Humidity",
    "WB_CCKP_ID": "Ice Days",
    "WB_CCKP_PR": "Precipitation",
    "WB_CCKP_R20MM": "Heavy Rain Days (≥20mm)",
    "WB_CCKP_R50MM": "Very Heavy Rain Days (≥50mm)",
    "WB_CCKP_R95PTOT": "Extreme Rain (95th percentile)",
    "WB_CCKP_RX1DAY": "Max 1-Day Precipitation",
    "WB_CCKP_RX5DAY": "Max 5-Day Precipitation",
    "WB_CCKP_SD": "Snow Days",
    "WB_CCKP_TAS": "Mean Temperature",
    "WB_CCKP_TASMAX": "Max Temperature",
    "WB_CCKP_TASMIN": "Min Temperature",
    "WB_CCKP_TNN": "Min of Daily Min Temperatures",
    "WB_CCKP_TR": "Tropical Nights",
    "WB_CCKP_TR23": "Tropical Nights >23°C",
    "WB_CCKP_TR26": "Tropical Nights >26°C",
    "WB_CCKP_TR29": "Tropical Nights >29°C",
    "WB_CCKP_TR32": "Tropical Nights >32°C",
    "WB_CCKP_TX84RR": "Max Temp on Rainy Days (84th %)",
    "WB_CCKP_TXX": "Max of Daily Max Temperatures",
    "WB_CCKP_WSDI": "Warm Spell Duration Index"
}

In [3]:
type(indicator_labels)

dict

In [4]:
# Load model
model = joblib.load("../models/xgboost.joblib")

# Feature names
feat_names = list(getattr(model, "feature_names_in_", X.columns))

# Importance values
imp = model.feature_importances_
fi = pd.DataFrame({"feature": feat_names, "importance": imp})
fi = fi.sort_values("importance", ascending=False, ignore_index=True)

# Map to readable labels
fi["feature_label"] = fi["feature"].map(indicator_labels).fillna(fi["feature"])

# Save CSV
outdir = "../results/xai"
os.makedirs(outdir, exist_ok=True)
fi.to_csv(os.path.join(outdir, "feature_importance.csv"), index=False)

# Plot top 20
fi_top = fi.head(20).iloc[::-1]
plt.figure(figsize=(10,6))
plt.barh(fi_top["feature_label"], fi_top["importance"])
plt.xlabel("Importance")
plt.title("XGBoost Feature Importance")
plt.tight_layout()
plt.savefig(os.path.join(outdir, "feature_importance.png"), dpi=200)
plt.close()

print("Saved feature importance results in:", outdir)

Saved feature importance results in: ../results/xai


In [5]:
import shap
import matplotlib.pyplot as plt
import numpy as np
import os

outdir = "../results/xai"
os.makedirs(outdir, exist_ok=True)

# Apply mapping to get readable labels
feat_names = list(getattr(model, "feature_names_in_", X.columns))
feat_labels = [indicator_labels.get(f, f) for f in feat_names]

# Make sure columns are aligned with model
X_aligned = X[feat_names]
X_np = X_aligned.to_numpy(dtype=np.float64, copy=True)

# Explainer
explainer = shap.TreeExplainer(model, X_np, feature_perturbation="interventional")
shap_values = explainer.shap_values(X_np)

# --- Bar plot (global importance) ---
plt.figure()
shap.summary_plot(shap_values, X_aligned, feature_names=feat_labels, plot_type="bar", show=False)
plt.tight_layout()
plt.savefig(os.path.join(outdir, "shap_bar.png"), dpi=200)
plt.close()

# --- Beeswarm plot (distribution of impacts) ---
plt.figure()
shap.summary_plot(shap_values, X_aligned, feature_names=feat_labels, show=False)
plt.tight_layout()
plt.savefig(os.path.join(outdir, "shap_beeswarm.png"), dpi=200)
plt.close()

print("Saved SHAP bar and beeswarm plots in:", outdir)

c:\Users\katsantouris\AppData\Local\anaconda3\envs\dissertation\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|===================| 2432/2436 [01:04<00:00]        

Saved SHAP bar and beeswarm plots in: ../results/xai


In [7]:
import lime
import lime.lime_tabular
import matplotlib.pyplot as plt
import os
import numpy as np

outdir = "../results/xai"
os.makedirs(outdir, exist_ok=True)

# Feature names + mapped labels
feat_names = list(getattr(model, "feature_names_in_", X.columns))
feat_labels = [indicator_labels.get(f, f) for f in feat_names]

# Initialize LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X[feat_names].to_numpy(dtype=np.float64),
    feature_names=feat_labels,
    mode="regression",
    verbose=False,
    random_state=42
)

# Crop indicators (adjust if your one-hot names differ)
crop_features = ["ITEM_Barley", "ITEM_Wheat", "ITEM_Maize"]

examples = {}
for crop in crop_features:
    # pick the first sample in test set belonging to this crop
    idx = X_test_features.index[X_test_features[crop] == 1][0]
    row = X_test_features.loc[idx, feat_names].to_numpy()
    
    exp = explainer.explain_instance(
        row,
        model.predict,
        num_features=10
    )
    
    pred = exp.predicted_value
    actual = y_test.loc[idx]
    
    # Save plot
    fig = exp.as_pyplot_figure()
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"lime_{crop}.png"), dpi=200)
    plt.close(fig)
    
    examples[crop] = {
        "index": idx,
        "predicted": pred,
        "actual": actual
    }

# Print summary
for crop, info in examples.items():
    print(f"{crop}: sample {info['index']}, predicted={info['predicted']:.2f}, actual={info['actual']:.2f}")


ITEM_Barley: sample 4, predicted=3637.58, actual=3033.20
ITEM_Wheat: sample 1, predicted=4026.45, actual=4500.10
ITEM_Maize: sample 0, predicted=9058.32, actual=10169.50
